In [2]:
import torch

In [3]:
import utils

In [4]:
device = utils.get_device()
device

'mps'

In [5]:
# 1. Load dataset

In [6]:
import os.path as op

from datasets import load_dataset

import lightning as L
from lightning.pytorch.loggers import CSVLogger
from lightning.pytorch.callbacks import ModelCheckpoint

import numpy as np
import pandas as pd
import torch

from sklearn.feature_extraction.text import CountVectorizer

from local_dataset_utilities import download_dataset, load_dataset_into_to_dataframe, partition_dataset
from local_dataset_utilities import IMDBDataset

In [17]:
download_dataset()

41% | 33.05 MB | 5.11 MB/s | 6.47 sec elapsed

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



69% | 55.75 MB | 6.54 MB/s | 8.52 sec elapsed

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



88% | 71.33 MB | 7.30 MB/s | 9.77 sec elapsed

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [18]:
df = load_dataset_into_to_dataframe()
partition_dataset(df)

100%|███████████████████████████████████| 50000/50000 [00:14<00:00, 3477.58it/s]


Class distribution:


In [10]:
len(df), df.sample(5)

(50000,
                                                 text  label
 0  Robert Duvall is a direct descendent of Confed...      1
 0  The Railway Children, at least this 1970 movie...      1
 0  Anatomie was a German made Movie and subtitled...      0
 0  This film makes Clooney. All his films combine...      1
 0  This is easily the best cinematic version of W...      1)

In [11]:
df['label'].value_counts()

1    25000
0    25000
Name: label, dtype: int64

In [15]:
ls

1-LLM-as-Feature-Extractor.ipynb  test.csv
__pycache__/                      train.csv
gitignore.txt                     utils.py
local_dataset_utilities.py        val.csv


In [16]:
df_train = pd.read_csv("train.csv")
df_val = pd.read_csv("val.csv")
df_test = pd.read_csv("test.csv")

In [19]:
# 2. Tokenization, Vectorization

In [20]:
# datasets.load_dataset()
imdb_dataset = load_dataset(
    "csv",
    data_files={
        "train": "train.csv",
        "validation": "val.csv",
        "test": "test.csv"
    }
)

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /Users/czq/.cache/huggingface/datasets/csv/default-2654090031c6f84b/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [21]:
from transformers import AutoTokenizer

In [22]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [23]:
print("Tokenizer max input length:", tokenizer.model_max_length)
print("Tokenizer vocab size:", tokenizer.vocab_size)

Tokenizer max input length: 512
Tokenizer vocab size: 30522


In [28]:
def tokenize_text(batch):
    return tokenizer(batch["text"], truncation=True, padding=True)

In [29]:
imdb_tokenized = imdb_dataset.map(tokenize_text, batched=True, batch_size=None)

Map:   0%|          | 0/35000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [31]:
del imdb_dataset

In [32]:
# 3. Using DistilBert as Feature Extractor

In [33]:
from transformers import AutoModel

In [34]:
model = AutoModel.from_pretrained("distilbert-base-uncased")
model.to(device)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


DistilBertModel(
  (embeddings): Embeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): Transformer(
    (layer): ModuleList(
      (0-5): 6 x TransformerBlock(
        (attention): MultiHeadSelfAttention(
          (dropout): Dropout(p=0.1, inplace=False)
          (q_lin): Linear(in_features=768, out_features=768, bias=True)
          (k_lin): Linear(in_features=768, out_features=768, bias=True)
          (v_lin): Linear(in_features=768, out_features=768, bias=True)
          (out_lin): Linear(in_features=768, out_features=768, bias=True)
        )
        (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (ffn): FFN(
          (dropout): Dropout(p=0.1, inplace=False)
          (lin1): Linear(in_features=768, out_features=3072, bias=True)
          (lin2): Li

In [35]:
imdb_tokenized

DatasetDict({
    train: Dataset({
        features: ['index', 'text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 35000
    })
    validation: Dataset({
        features: ['index', 'text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['index', 'text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 10000
    })
})

In [36]:
imdb_tokenized.set_format("torch", columns=["input_ids", "attention_mask", "label"])

In [38]:
keys = ["attention_mask", "input_ids"]
test_batch = {key: imdb_tokenized["train"][:3][key].to(device) for key in keys}

In [39]:
with torch.inference_mode():
    test_output = model(**test_batch)

In [40]:
test_output.last_hidden_state.shape

torch.Size([3, 512, 768])

In [41]:
cls_token_output = test_output.last_hidden_state[:,0,:]
cls_token_output.shape

torch.Size([3, 768])

In [44]:
@torch.inference_mode()
def get_output_embeddings(batch):
    output = model(
        batch["input_ids"].to(device),
        attention_mask=batch["attention_mask"].to(device)).last_hidden_state[:, 0]
    return {"features": output.cpu().numpy()}

In [45]:
imdb_features = imdb_tokenized.map(get_output_embeddings, batched=True, batch_size=10)

Map:   0%|          | 0/35000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [46]:
imdb_features

DatasetDict({
    train: Dataset({
        features: ['index', 'text', 'label', 'input_ids', 'attention_mask', 'features'],
        num_rows: 35000
    })
    validation: Dataset({
        features: ['index', 'text', 'label', 'input_ids', 'attention_mask', 'features'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['index', 'text', 'label', 'input_ids', 'attention_mask', 'features'],
        num_rows: 10000
    })
})

In [53]:
x_train = np.array(imdb_features["train"]["features"])
y_train = np.array(imdb_features["train"]["label"])

x_val = np.array(imdb_features["validation"]["features"])
y_val = np.array(imdb_features["validation"]["label"])

x_test = np.array(imdb_features["test"]["features"])
y_test = np.array(imdb_features["test"]["label"])

(x_train.shape, y_train.shape), (x_val.shape, y_val.shape), (x_test.shape, y_test.shape)

(((35000, 768), (35000,)), ((5000, 768), (5000,)), ((10000, 768), (10000,)))

In [54]:
# 4. Train model on extracted features (i.e. embeddings)

In [55]:
from sklearn.linear_model import LogisticRegression

In [56]:
clf = LogisticRegression(max_iter=1000)

In [57]:
clf.fit(x_train, y_train)

/Users/czq/miniforge3/envs/torch/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=1000)

In [60]:
print("Training accuracy", clf.score(x_train, y_train))
print("Validation accuracy", clf.score(x_val, y_val))
print("Test accuracy", clf.score(x_test, y_test))

Training accuracy 0.8868571428571429
Validation accuracy 0.8836
Test accuracy 0.8795
